In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

class ThalexDataLoader:
    def __init__(self, metrics_dir="/home/aladhimarkets/Thalex_SimpleQuouter/metrics"):
        """Initialize with path to metrics directory"""
        self.metrics_dir = metrics_dir
        self.performance_file = os.path.join(metrics_dir, "performance_metrics.csv")
        self.trade_file = os.path.join(metrics_dir, "trade_history.csv")
        self.quote_file = os.path.join(metrics_dir, "quote_metrics.csv")
        
        # Load data files
        self.performance_df = None
        self.trades_df = None
        self.quotes_df = None
        self.load_data()
    
    def load_data(self):
        """Load data from CSV files"""
        # Check if files exist
        files_exist = {
            "performance_metrics": os.path.exists(self.performance_file),
            "trade_history": os.path.exists(self.trade_file),
            "quote_metrics": os.path.exists(self.quote_file)
        }
        
        print("Available metrics files:")
        for name, exists in files_exist.items():
            print(f"- {name}: {'Found' if exists else 'Not found'}")
        
        # Load the data if files exist
        try:
            if files_exist["performance_metrics"]:
                self.performance_df = pd.read_csv(
                    self.performance_file,
                    on_bad_lines='skip',
                    engine='python'
                )
                self.performance_df['timestamp'] = pd.to_datetime(self.performance_df['timestamp'])
                
                # Calculate total PnL
                if 'realized_pnl' in self.performance_df.columns and 'unrealized_pnl' in self.performance_df.columns:
                    self.performance_df['total_pnl'] = self.performance_df['realized_pnl'] + self.performance_df['unrealized_pnl']
                
                print(f"Performance metrics: {len(self.performance_df)} records")
                if len(self.performance_df) > 0:
                    print(f"Date range: {self.performance_df['timestamp'].min()} to {self.performance_df['timestamp'].max()}")
        except Exception as e:
            print(f"Error loading performance metrics: {str(e)}")
            self.performance_df = pd.DataFrame()
            
        try:
            if files_exist["trade_history"]:
                self.trades_df = pd.read_csv(
                    self.trade_file,
                    on_bad_lines='skip',
                    engine='python'
                )
                if 'timestamp' in self.trades_df.columns:
                    self.trades_df['timestamp'] = pd.to_datetime(self.trades_df['timestamp'])
        except Exception as e:
            print(f"Error loading trade history: {str(e)}")
            self.trades_df = pd.DataFrame()
        
        try:
            if files_exist["quote_metrics"]:
                self.quotes_df = pd.read_csv(
                    self.quote_file,
                    on_bad_lines='skip',
                    engine='python'
                )
                if 'timestamp' in self.quotes_df.columns:
                    self.quotes_df['timestamp'] = pd.to_datetime(self.quotes_df['timestamp'])
        except Exception as e:
            print(f"Error loading quote metrics: {str(e)}")
            self.quotes_df = pd.DataFrame()
    
    def get_available_metrics(self):
        """Get list of available metrics"""
        if self.performance_df is not None:
            return list(self.performance_df.columns)
        return []

def create_performance_dashboard(data_loader):
    """Create a comprehensive trading performance dashboard using actual data"""
    if data_loader.performance_df is None or len(data_loader.performance_df) == 0:
        print("No performance data available to display")
        return None
    
    df = data_loader.performance_df
    
    # Display available metrics
    print("Available metrics:", data_loader.get_available_metrics())
    
    # Create subplot structure
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            "Price & Position", "P&L", 
            "Volatility", "Spread", 
            "Z-Score", "Quote Depth",
            "Trade Distribution", "Risk Metrics"
        ),
        vertical_spacing=0.1,
        horizontal_spacing=0.1,
        specs=[
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "bar"}, {"type": "indicator"}]
        ]
    )
    
    # 1. Price & Position chart
    if 'mark_price' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=df['mark_price'], name='Price', line=dict(color='blue')),
            row=1, col=1
        )
    
    if 'position_size' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=df['position_size'], name='Position', line=dict(color='green', dash='dash')),
            row=1, col=1
        )
    
    # 2. PnL chart (improved)
    # Add realized PnL
    if 'realized_pnl' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'],
                y=df['realized_pnl'],
                name='Realized PnL',
                line=dict(color='green')
            ),
            row=1, col=2
        )
    
    # Add unrealized PnL
    if 'unrealized_pnl' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'],
                y=df['unrealized_pnl'],
                name='Unrealized PnL',
                line=dict(color='blue')
            ),
            row=1, col=2
        )
    
    # Add total PnL
    if 'total_pnl' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'],
                y=df['total_pnl'],
                name='Total PnL',
                line=dict(color='purple', dash='dash')
            ),
            row=1, col=2
        )
    
    # Add zero line for PnL reference
    fig.add_hline(
        y=0,
        line_dash="dot",
        line_color="gray",
        opacity=0.5,
        row=1, col=2
    )
    
    # 3. Volatility chart
    if 'volatility' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=df['volatility'], name='Volatility', line=dict(color='red')),
            row=2, col=1
        )
    
    # 4. Spread chart (improved)
    if 'best_bid' in df.columns and 'best_ask' in df.columns:
        # Plot bid and ask prices
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'],
                y=df['best_bid'],
                name='Best Bid',
                line=dict(color='green', width=1)
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=df['timestamp'],
                y=df['best_ask'],
                name='Best Ask',
                line=dict(color='red', width=1)
            ),
            row=2, col=2
        )
        
        # Plot spread using bid_ask_spread column
        if 'bid_ask_spread' in df.columns:
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'],
                    y=df['bid_ask_spread'],
                    name='Spread',
                    line=dict(color='orange', dash='dot'),
                    yaxis='y5'  # Use a separate y-axis for spread
                ),
                row=2, col=2
            )
    
    # 5. Z-Score chart
    if 'zscore' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=df['zscore'], name='Z-Score', line=dict(color='black')),
            row=3, col=1
        )
        # Add threshold lines
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=[2]*len(df), name='Upper Threshold', line=dict(color='red', dash='dash')),
            row=3, col=1
        )
        fig.add_trace(
            go.Scatter(x=df['timestamp'], y=[-2]*len(df), name='Lower Threshold', line=dict(color='red', dash='dash')),
            row=3, col=1
        )
    
    # 6. Quote Depth visualization
    if data_loader.quotes_df is not None and len(data_loader.quotes_df) > 0:
        quotes = data_loader.quotes_df
        if 'side' in quotes.columns and 'level' in quotes.columns:
            # Group by timestamp and side
            latest_quotes = quotes.sort_values('timestamp').groupby(['side', 'level']).last().reset_index()
            
            # Add to chart
            for side, color in [('bid', 'green'), ('ask', 'red')]:
                side_quotes = latest_quotes[latest_quotes['side'] == side]
                if len(side_quotes) > 0:
                    fig.add_trace(
                        go.Bar(
                            x=side_quotes['level'],
                            y=side_quotes['amount'] if 'amount' in side_quotes.columns else [1] * len(side_quotes),
                            name=f"{side} quotes",
                            marker_color=color
                        ),
                        row=3, col=2
                    )
    
    # 7. Trade Distribution
    if data_loader.trades_df is not None and len(data_loader.trades_df) > 0:
        trades = data_loader.trades_df
        if 'direction' in trades.columns:
            direction_counts = trades['direction'].value_counts()
            fig.add_trace(
                go.Bar(
                    x=direction_counts.index,
                    y=direction_counts.values,
                    name="Trade Direction"
                ),
                row=4, col=1
            )
    
    # 8. Risk Metrics indicator
    current_pnl = 0
    previous_pnl = 0
    if 'total_pnl' in df.columns and len(df) > 0:
        current_pnl = df['total_pnl'].iloc[-1]
        previous_pnl = df['total_pnl'].iloc[-2] if len(df) > 1 else 0
    
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=current_pnl,
            title={"text": "Current P&L"},
            delta={'reference': previous_pnl, 'relative': True, 'valueformat': '.2%'},
            number={'valueformat': '.2f'}
        ),
        row=4, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1000,
        width=1200,
        title_text="Avellaneda-Stoikov Trading Performance Dashboard",
        showlegend=True
    )
    
    # Add secondary y-axis for spread
    fig.update_layout(
        yaxis5=dict(
            title="Spread",
            anchor="x",
            overlaying="y4",  # Overlay on the price axis
            side="right"
        )
    )
    
    return fig

# Load the data and create dashboard
data_loader = ThalexDataLoader()
fig = create_performance_dashboard(data_loader)

# Display the dashboard
if fig is not None:
    fig.show()

Available metrics files:
- performance_metrics: Found
- trade_history: Found
- quote_metrics: Found
Performance metrics: 1770 records
Date range: 2025-03-21 16:00:31.490142 to 2025-03-24 11:59:35.912606
Available metrics: ['timestamp', 'mark_price', 'index_price', 'position_size', 'position_value', 'entry_price', 'position_start_time', 'realized_pnl', 'unrealized_pnl', 'highest_profit', 'max_drawdown', 'current_drawdown', 'position_utilization', 'notional_utilization', 'win_count', 'loss_count', 'consecutive_losses', 'breach_position_size', 'breach_notional_value', 'breach_drawdown', 'breach_loss_streak', 'order_total_orders', 'order_filled_orders', 'order_cancelled_orders', 'order_amended_orders', 'order_rejected_orders', 'volatility', 'zscore', 'is_trending', 'is_volatile', 'trend_strength', 'market_impact', 'order_fill_rate', 'order_cancel_rate', 'order_reject_rate', 'best_bid', 'best_ask', 'bid_ask_spread', 'mid_price', 'total_pnl']
